<a href="https://colab.research.google.com/github/mightyoctopus/hugging_face_quantization/blob/main/w3_d4_quantization_actual_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U -q transformers bitsandbytes accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 121.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.7/374.7 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 121.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 97.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
from google.colab import userdata, drive
from huggingface_hub import login, snapshot_download
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TextStreamer, AutoConfig
import torch

In [3]:
cache_path = "/content/drive/MyDrive/Colab Notebooks/huggingface_cache"

In [4]:
hf_token = userdata.get("HF_TOKEN")
login(hf_token, add_to_git_credential=True)

In [5]:
### Instruct Models:

LLAMA = "meta-llama/Llama-3.1-8B-Instruct"
PHI3 = "microsoft/Phi-3-mini-4k-instruct"
GEMMA2 = "google/gemma-2-2b-it"
QWEN2 = "Qwen/Qwen2-7B-Chat"
MIXTRAL = "mistralai/Mixtral-8x7B-Instruct-v0.1" # Use smaller if out of memory.

In [6]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Tell a light-hearted joke for a room of Data Scientists"}
]

In [7]:
bnb_config_4bit = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

In [8]:
tokenizer = AutoTokenizer.from_pretrained(LLAMA, cache_dir=cache_path)
tokenizer.pad_token = tokenizer.eos_token
inputs = tokenizer.apply_chat_template(
    messages,
    return_tensors="pt",
    tokenize=True
).to("cuda")
print(inputs)

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

tensor([[128000, 128006,   9125, 128007,    271,  38766,   1303,  33025,   2696,
             25,   6790,    220,   2366,     18,    198,  15724,   2696,     25,
            220,   1627,  10263,    220,   2366,     19,    271,   2675,    527,
            264,  11190,  18328,     13, 128009, 128006,    882, 128007,    271,
          41551,    264,   3177,  70395,  22380,    369,    264,   3130,    315,
           2956,  57116, 128009]], device='cuda:0')


In [9]:
model = AutoModelForCausalLM.from_pretrained(
    LLAMA,
    device_map="cuda",
    quantization_config=bnb_config_4bit,
    cache_dir=cache_path
)

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [10]:
memory = model.get_memory_footprint() / (1024 ** 2)
print(f"{memory:,.1f} MB")

5,332.5 MB


In [11]:
outputs = model.generate(
    inputs,
    max_new_tokens=80
    )

print("RESULT: ", tokenizer.decode(outputs[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


RESULT:  <|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are a helpful assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>

Tell a light-hearted joke for a room of Data Scientists<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Why did the regression model go to therapy?

Because it was struggling to generalize its feelings.<|eot_id|>


In [27]:
### Clean up cache
# del inputs, outputs, model
# torch.cuda.empty_cache()

In [13]:
### Wrapping everything above into a modular function:

def generate(model_name, messages):

  # Manually download to Drive
  model_path = snapshot_download(
    repo_id=model_name,
    cache_dir=cache_path,
    local_dir_use_symlinks=False  # safer in Drive
  )

  tokenizer = AutoTokenizer.from_pretrained(model_path)

  tokenizer.pad_token = tokenizer.eos_token
  inputs = tokenizer.apply_chat_template(
      messages,
      tokenize=True,
      return_tensors="pt"
  ).to("cuda")

  streamer = TextStreamer(tokenizer)

  model = AutoModelForCausalLM.from_pretrained(
      model_path,
      device_map="auto",
      quantization_config=bnb_config_4bit,
  )

  outputs = model.generate(
      inputs,
      max_new_tokens=80,
      streamer=streamer,
  )

  del tokenizer, model, inputs, outputs
  torch.cuda.empty_cache()

In [15]:
generate(LLAMA, messages)

Fetching 17 files:   0%|          | 0/17 [00:00<?, ?it/s]

original/consolidated.00.pth:   0%|          | 0.00/16.1G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/44.0k [00:00<?, ?B/s]

original/tokenizer.model:   0%|          | 0.00/2.18M [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

...

params.json:   0%|          | 0.00/199 [00:00<?, ?B/s]

LICENSE:   0%|          | 0.00/7.63k [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are a helpful assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>

Tell a light-hearted joke for a room of Data Scientists<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Why did the logistic regression model go to therapy?

Because it was struggling to classify its emotions.<|eot_id|>


In [28]:
import json

# Path to your notebook file in Colab
notebook_path = "/content/hugging_face_quantization/hugging_face_quantization/w3_d4_quantization_actual_.ipynb"

# Load notebook JSON
with open(notebook_path, "r", encoding="utf-8") as f:
    nb = json.load(f)

# Remove the top-level widgets metadata
if "widgets" in nb.get("metadata", {}):
    del nb["metadata"]["widgets"]

# Remove widgets metadata inside each cell (just in case)
for cell in nb.get("cells", []):
    if "widgets" in cell.get("metadata", {}):
        del cell["metadata"]["widgets"]

# Save back the cleaned notebook
with open(notebook_path, "w", encoding="utf-8") as f:
    json.dump(nb, f, ensure_ascii=False, indent=1)

print(f"✅ Cleaned notebook saved: {notebook_path}")


✅ Cleaned notebook saved: /content/hugging_face_quantization/hugging_face_quantization/w3_d4_quantization_actual_.ipynb
